# Segmenting and Clustering Neighborhoods in Toronto - Assignment

## Step 1 - web scraping of Neighborhood data of Toronto

In [29]:
# import libraries
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import string

In [30]:
# get web page text
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [31]:
# get postal code, borough, neighborhood values from table in the web page
summary = soup.find('table',class_='wikitable sortable')

In [32]:
# define dataframe
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


In [33]:
# initialize variables
postal_row_count = 1
prev_postal_code = ''
prev_borough_name = ''
c_neighborhood_name = ''

In [34]:
# populate dataframe with data from the web page table
for tr in summary.find_all('tr')[1:]:
    tds = tr.find_all('td')
    t_postal_code = tds[0].text
    borough_name = tds[1].text
    neighborhood_name = tds[2].text    
    
    if neighborhood_name == 'Not assigned\n' and borough_name != 'Not assigned' :
        neighborhood_name = borough_name
        print(t_postal_code)
        print(neighborhood_name)
        
    if (t_postal_code == prev_postal_code) : 
        prev_postal_code = t_postal_code
        c_neighborhood_name = c_neighborhood_name + ',' + neighborhood_name
    elif postal_row_count == 1:
        postal_row_count = 0
        c_neighborhood_name = neighborhood_name
        prev_postal_code = t_postal_code
        prev_borough_name = borough_name
    else:    
        neighborhoods = neighborhoods.append({'PostalCode':prev_postal_code, 
                                          'Borough': prev_borough_name,
                                          'Neighborhood': c_neighborhood_name}, ignore_index=True)
        prev_postal_code = t_postal_code
        prev_borough_name = borough_name
        c_neighborhood_name = neighborhood_name

M7A
Queen's Park


In [35]:
# examine the resulting dataframe
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace('\n','')
neighborhoods.head(10) 

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Rouge,Malvern"


In [36]:
# remove boroughs with 'Not assigned' values
neighborhoods.drop(neighborhoods[neighborhoods['Borough']== 'Not assigned'].index, inplace=True)
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park


In [37]:
# get shape of the data frame
neighborhoods.shape

(103, 3)